<a href="https://colab.research.google.com/github/ITA-LOW/projeto_senex/blob/main/Data_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ITA-LOW/projeto_senex/main/banco%20de%20dados/data_frame_p%C3%B3s_ETL.csv')
df.drop(columns=['Unnamed: 0','ESTADO','ANO'], inplace = True)

In [3]:
df

,IDADE,SEXO,RACACOR,ESTCIV,CAUSABAS
0,91,1,1,2,C61
1,80,1,1,2,I500
2,79,2,1,3,I500
3,63,1,1,2,C710
4,68,2,1,2,C349
...,...,...,...,...,...
208526,83,2,1,3,I429
208527,66,1,4,1,K769
208528,84,1,1,2,K921
208529,85,2,1,3,I219


In [4]:
df_teste = df.iloc[0:10001]
df_teste

,IDADE,SEXO,RACACOR,ESTCIV,CAUSABAS
0,91,1,1,2,C61
1,80,1,1,2,I500
2,79,2,1,3,I500
3,63,1,1,2,C710
4,68,2,1,2,C349
...,...,...,...,...,...
9996,74,2,1,3,C260
9997,61,1,3,2,C169
9998,71,1,2,2,I10
9999,78,2,1,3,C169


In [5]:
X = df_teste.iloc[:, 0:-1].values #features: da primeira até a penúltima coluna
#y = df_teste.iloc[:, -1].values #rótulo: última coluna
y = pd.DataFrame(df_teste.iloc[:, -1].values,columns=['CAUSABAS'])

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = y.apply(le.fit_transform)
y

,CAUSABAS
0,139
1,402
2,402
3,147
4,99
...,...
9996,88
9997,60
9998,347
9999,60


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = 1/3, 
                                                    random_state = 42)

In [8]:
X_train_2, X_val, y_train_2, y_val = train_test_split(X_train,
                                                      y_train, 
                                                      test_size = 1/3, 
                                                      random_state = 42)

In [9]:
# Feature Scaling: alguns algoritmos são sensíveis à escala
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train_2)
X_val = scaler.transform(X_val)
#é possivel testar outras formas de normalização

In [10]:
#Dicionário de resultados
dic = {}

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
# Treinando regressão logística
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 42)
lr.fit(X_train_2, y_train_2)
dic["Regressão Logística"] =  accuracy_score(y_val, lr.predict(X_val))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [13]:
# Treinando o k-NN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train_2, y_train_2)
dic["kNN_7"] =  accuracy_score(y_val, knn.predict(X_val))

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [14]:
#Treinando uma árvore de decisão
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 50, random_state = 42)
tree.fit(X_train_2, y_train_2)
dic["Decision_tree_4"] = accuracy_score(y_val, tree.predict(X_val))

In [15]:
#Treinando uma floresta aleatória
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators= 100, 
                            random_state = 42)
rf.fit(X_train_2, y_train_2)
dic["Random_forest_100"] = accuracy_score(y_val, rf.predict(X_val))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [16]:
# Treinando SVM
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(X_train_2, y_train_2)
dic["SVC"] = accuracy_score(y_val, svc.predict(X_val))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
# Treinando rede neural
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(3,3,3,3),max_iter=3000,random_state=42)
mlp.fit(X_train_2, y_train_2)
dic["Rede Neural"] = accuracy_score(y_val, mlp.predict(X_val))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
validation = pd.Series(dic, name="Acurácia")
validation

Regressão Logística    0.085470
kNN_7                  0.026991
Decision_tree_4        0.053081
Random_forest_100      0.050832
SVC                    0.086370
Rede Neural            0.077373
Name: Acurácia, dtype: float64

In [19]:
#Treinando o melhor modelo no conjunto inteiro:
scaler = MinMaxScaler()    
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  

In [20]:
#Decision Tree não foi o melhor, é só um exemplo
dt = DecisionTreeClassifier(random_state = 42)     
dt.fit(X_train, y_train)
print("Acurácia final: ", str(accuracy_score(y_test, dt.predict(X_test))))

Acurácia final:  0.05158968206358728


# Como os resultados foram horríveis será necessário rever a forma de se apresentar o target